First we need to get our predicted image from our model and use the predected polygons to extract our rooftops

In [46]:
%%capture
!pip install geopandas
!pip install rasterio

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import fiona
import geopandas as gpd
import rasterio as rio
from PIL import Image
import rasterio.mask

In [85]:
gdf_path = '/content/drive/MyDrive/Official Folder Omdena Talent Good3Energy/Pipeline/output/new_geo_px.geojson'
img_path = '/content/drive/MyDrive/Official Folder Omdena Talent Good3Energy/Pipeline/tile_image/building_tile.png'

im = Image.open(img_path)
im.save('building_tile.tif')

def mask_buildings(raster_path, geoson_path, mask_path):
    '''
    Takes the paths for a raster image  and a geoJson mask
    '''
    # Get the geoJson mask and the raster image
    with fiona.open(geoson_path, 'r') as gdf:
         polys = [feature["geometry"] for feature in gdf]


    with rio.open(raster_path) as src:
        out_image, out_transform = rio.mask.mask(src, polys, crop=True)
        out_meta = src.meta
    
    # Save the new masked image    
    out_meta.update({ "driver": "PNG",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})

    with rio.open(mask_path, "w", **out_meta) as dest:
        dest.write(out_image)


mask_buildings('building_tile.tif', gdf_path, 'masked.png')

/usr/local/lib/python3.6/dist-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.6/dist-packages/rasterio/__init__.py:223: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  **kwargs)


In [86]:
# We can use our previous code to extract each building 

import json
from PIL import Image
def get_bboxes(coco_file):
    '''Get the bbox list from our prediction json'''
    # Read the coco file and get the boxes
    ## Read and parse our json file
    with open(coco_file, 'r') as my_file:
        data = my_file.readlines()
        # Parse file
        obj = json.loads(data[0])
        obj = json.loads(obj)
    bbox_list = []
    for box in obj['boxes']:
        bbox_list.append(box)
    return bbox_list



def extract_buildings(bbox_list, img, start_index):
    '''Extract the buildings based on our model prediction bounding boxes'''   
    for index, box in enumerate(bbox_list, start=start_index):
        im=Image.open(img)
        im=im.crop(box)
        # Save the image 
        im.save('/content/drive/MyDrive/Official Folder Omdena Talent Good3Energy/Edge_detection/extracted_roofs/' + 'building_'+str(index)+'.png')

        
# Get our boxes
boxes = get_bboxes('/content/drive/MyDrive/Official Folder Omdena Talent Good3Energy/Pipeline/output/final_pred.json')
image = '/content/masked.png'

# Extract and save our buildings (start_index is for naming purposes)
extract_buildings(boxes, image, start_index=0)



Reference:
- https://stackoverflow.com/questions/30901019/extracting-polygon-given-coordinates-from-an-image-using-opencv